This notebook plots some additional plots for the poster.

Let's start by plotting VaR and CVaR of a simple distribution.

In [18]:
import numpy as np
from scipy.stats import gamma
%matplotlib
import matplotlib.pyplot as plt

alpha = 0.7

test_X = np.linspace(0, 5, 1001).reshape(-1)
rv = gamma(a=2., scale=0.5)
pdf = rv.pdf(test_X)

plt.plot(test_X, pdf, label="probability density")
plt.grid(True)

scaled_pdf = pdf / np.sum(pdf)

var_index = 0
sum = 0.
while True:
    if sum > alpha:
        break
    sum += scaled_pdf[var_index]
    var_index += 1

var_X = test_X[var_index]

cvar_weights = pdf[var_index:] / np.sum(pdf[var_index:])
cvar = np.sum(test_X[var_index:] * cvar_weights)


plt.fill_between(test_X[var_index:], 0, pdf[var_index:], color="pink", alpha=0.5)
plt.axvline(var_X, label="Value-at-Risk", color="tab:green", lw=3.)
plt.axvline(cvar, label="Conditional Value-at-Risk", color="tab:orange", lw=3.)
plt.title(f"VaR and CVaR at risk level $\\alpha={alpha}$")
plt.legend()
plt.ylim(0., 0.75)

plt.savefig("gamma_var_cvar.pdf", dpi=300)

Using matplotlib backend: TkAgg


Let's plot the VaR and CVaR of a GP sample path.

In [63]:
import torch
from botorch.models import SingleTaskGP
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.fit import fit_gpytorch_model

num_train = 20
dim = 1
model = SingleTaskGP(torch.rand(num_train, dim), torch.randn(num_train, 1))
mll = ExactMarginalLogLikelihood(model.likelihood, model)
_ = fit_gpytorch_model(mll)

num_test = 1001
test_X = torch.linspace(0, 1, num_test).reshape(-1, 1)
with torch.no_grad():
    pm = model.posterior(test_X).mean

plt.close()
plt.plot(test_X, pm, label="GP sample")

ordered_pm, _ = torch.sort(pm, dim=0)
var = ordered_pm[int(alpha * num_test)]
cvar = ordered_pm[int(alpha * num_test):].mean()

plt.axhline(var, label="Value-at-Risk", color="tab:green", lw=3.)
plt.axhline(cvar, label="Conditional Value-at-Risk", color="tab:orange", lw=3.)

# plt.fill_between(test_X[pm >= var], var, pm[pm >= var], color="pink", alpha=0.5)
min_x = None
max_len = test_X[pm >= var].shape[0]
standard_dist = 1.1/(num_test-1) # has some tolerance just in case
for i, x in enumerate(test_X[pm >= var]):
    if min_x is None:
        min_x = x
    if i+1 == max_len or test_X[pm >= var][i+1] - x > standard_dist:
        plt.axvspan(min_x, x, color="pink", alpha=0.5)
        min_x = None

plt.title(f"VaR and CVaR of a GP sample at $\\alpha={alpha}$")
plt.grid(True)
plt.legend()

plt.savefig("gp_var_cvar.pdf", dpi=300)

/Users/saitcakmak/.conda/envs/BoRisk/lib/python3.7/site-packages/botorch/models/utils.py:189: InputDataWarning: Input data is not standardized. Please consider scaling the input to zero mean and unit variance.
  warnings.warn(msg, InputDataWarning)
